In [28]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import gensim
from gensim.models import Word2Vec
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords

In [44]:
path_to_json = 'full_format_recipes.json'
data = pd.read_json(path_to_json)
data.replace(r'^\s*$', np.nan, regex=True, inplace=True)  # Reemplazar cadenas vacías por NaN
data_clean = data.dropna()
filtered_data = data_clean[data_clean['rating'] > 4.9]
print(filtered_data)

                                              directions   fat  \
19     [Puree first 5 ingredients in blender until sm...  33.0   
53     [Preheat oven to 400°F. Melt 1/4 cup butter in...  87.0   
62     [Melt butter in heavy large skillet over mediu...  15.0   
83     [Preheat oven to 350°. Spread pistachios on a ...  18.0   
105    [Set up a grill for indirect grilling, with on...  10.0   
...                                                  ...   ...   
20099  [Preheat oven to 425°F. Mix garlic, 2 Tbsp. di...  32.0   
20109  [Preheat oven to 375°F. Unroll pie crust onto ...   7.0   
20111  [Bring water and saffron just to boil in heavy...  22.0   
20114  [Mix whiskey, maple syrup, bitters, and 1 tsp....   0.0   
20121  [1. Preheat the oven to 400°F. Spray a baking ...  70.0   

                                  date  \
19           2009-03-04 04:00:00+00:00   
53           2008-04-02 18:27:00+00:00   
62           2004-08-20 04:00:00+00:00   
83           2013-05-09 04:00:00+00:00 

In [51]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
from transformers import pipeline
import os

# ===============================
# 1. Preparar los datos
# ===============================


# Crear texto de entrada y salida
data_clean['input_text'] = "Título: " + data_clean['title']
data_clean['output_text'] = "Descripción: " + data_clean['desc'] + "\nDirections: " + data_clean['directions'].apply(lambda x: " ".join(x))

# Guardar los datos en un archivo de texto para entrenamiento
train_data = data_clean[['input_text', 'output_text']]
train_data_formatted = train_data.apply(lambda row: f"{row['input_text']} {row['output_text']}", axis=1)

# Guardar en un archivo plano
train_file_path = "train_recipes.txt"
train_data_formatted.to_csv(train_file_path, index=False, header=False)

print("Datos preparados y guardados en 'train_recipes.txt'.")

# ===============================
# 2. Fine-tuning de GPT-2
# ===============================

# Función para cargar datos de texto
def load_dataset(file_path, tokenizer, block_size=512):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

# Configuración del tokenizer y modelo
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Cargar el dataset
dataset = load_dataset(train_file_path, tokenizer)

# Configurar el collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,  # Menos épocas para entrenar más rápido
    per_device_train_batch_size=8,  # Tamaño de batch mayor si la memoria lo permite
    save_steps=1000,  # Guardar menos frecuentemente
    save_total_limit=1,  # Mantener solo 1 checkpoint para ahorrar espacio
    logging_dir="./logs",
    logging_steps=500  # Registrar menos frecuentemente
)


# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# Entrenar el modelo
print("Iniciando el entrenamiento...")
trainer.train()

# Guardar el modelo entrenado
output_dir = "./fine_tuned_gpt2_recipes"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Modelo entrenado y guardado en {output_dir}.")

# ===============================
# 3. Generación de nuevas recetas
# ===============================

# Cargar el modelo entrenado
print("Cargando el modelo entrenado...")
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

# Generar texto a partir de un título
def generar_receta(titulo, max_length=150):
    prompt = f"Título: {titulo}\nDescripción:"
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        inputs, 
        max_length=max_length, 
        num_return_sequences=1, 
        do_sample=True, 
        temperature=0.7
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Pedir un título por teclado y generar receta
if __name__ == "__main__":
    titulo = input("Introduce el título de la receta: ")
    print("\nGenerando la descripción y directions...\n")
    resultado = generar_receta(titulo)
    print(resultado)

C:\Users\casa\AppData\Local\Temp\ipykernel_8272\750165244.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['input_text'] = "Título: " + data_clean['title']
C:\Users\casa\AppData\Local\Temp\ipykernel_8272\750165244.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['output_text'] = "Descripción: " + data_clean['desc'] + "\nDirections: " + data_clean['directions'].apply(lambda x: " ".join(x))


Datos preparados y guardados en 'train_recipes.txt'.


C:\Users\casa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
  0%|          | 0/3 [27:18<?, ?it/s]


Iniciando el entrenamiento...


                                                     
 70%|███████   | 500/712 [1:56:41<45:21, 12.83s/it] 

{'loss': 2.4138, 'grad_norm': 2.9689009189605713, 'learning_rate': 1.4887640449438203e-05, 'epoch': 0.7}


                                                   
100%|██████████| 712/712 [2:46:27<00:00, 14.03s/it]   


{'train_runtime': 9987.9507, 'train_samples_per_second': 0.57, 'train_steps_per_second': 0.071, 'train_loss': 2.373061276553722, 'epoch': 1.0}
Modelo entrenado y guardado en ./fine_tuned_gpt2_recipes.
Cargando el modelo entrenado...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Generando la descripción y directions...

Título: Chocolate brownie
Descripción: The first thing I'd like to do is to make a chocolate brownie that's been assembled with very little effort. I'm not a foodie, and I don't enjoy making things on my own. But you can imagine how much I love chocolate brownies. The chocolate will be tender and fluffy, the flavor will be crisp and golden, and the texture will be creamy and flavorful.
Directions: Pulse together all ingredients for the brownie in a food processor until smooth. Pulse in 3/4 cup granulated sugar. (Sugar is a great addition in this recipe.) Add 1 cup granulated sugar to the brownie mixture and blend until smooth. Pour


In [53]:

# ===============================
# 3. Generación de nuevas recetas
# ===============================

# Cargar el modelo entrenado
print("Cargando el modelo entrenado...")
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

# Generar texto a partir de un título
def generar_receta(titulo, max_length=150):
    prompt = f"Título: {titulo}\nDescripción:"
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        inputs, 
        max_length=max_length, 
        num_return_sequences=1, 
        do_sample=True, 
        temperature=0.7
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Pedir un título por teclado y generar receta
if __name__ == "__main__":
    titulo = input("Introduce el título de la receta: ")
    print("\nGenerando la descripción y directions...\n")
    resultado = generar_receta(titulo)
    print(resultado)

Cargando el modelo entrenado...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generando la descripción y directions...

Título: pesto rice
Descripción: Can be prepared in 45 minutes or less.
Directions: Heat oil in heavy large pot over medium-high. Add garlic and sauté until fragrant, about 6 minutes. Add shallots and sauté until fragrant and beginning to brown, about 6 minutes. Add rice and toss to combine. Add basil and sauté until thickened, about 2 minutes. Add pesto and cook, stirring occasionally, until pesto is tender, about 2 minutes. Add remaining grains and stir 1 minute. Add rice and stir 1 minute. Season with salt and pepper. Cover and chill, covered. Preheat oven to 400°F. Brush rice with oil


In [37]:
from transformers import pipeline
import pandas as pd

# Cargar el modelo de traducción (inglés a español)
translator = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")
counter = 0

# Función para traducir texto

def translate_text(text):
    global counter
    counter += 1
    print(counter)
    """
    Traduce el texto al español usando el modelo Helsinki-NLP.

    Args:
        text (str): Texto original en inglés.

    Returns:
        str: Texto traducido al español.
    """
    try:
        translation = translator(text, truncation=True)
        print(f"Traducción generada: {translation[0]['translation_text']}")  # Imprimir la traducción generada

        return translation[0]['translation_text']
    except Exception as e:
        print(f"Error al traducir: {e}")
        return text  # Devuelve el texto original en caso de error

# Cargar los datos
# Asegúrate de que `filtered_data` contiene las columnas necesarias ('directions')
# Aquí asumimos que ya tienes un DataFrame llamado `filtered_data`

# Preprocesar y convertir listas a cadenas de texto
filtered_data['directions_str'] = filtered_data['directions'].dropna().apply(lambda x: " ".join(x))  # Convertir listas a strings

# Aplicar la función de traducción a las instrucciones
filtered_data['directions_translated'] = filtered_data['directions_str'].apply(translate_text)

# Exportar a un archivo CSV
filtered_data[['directions_translated']].to_csv('directions_traducidas.csv', index=False)
print("Traducciones exportadas a 'directions_traducidas.csv'")


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
C:\Users\casa\AppData\Local\Temp\ipykernel_8272\947809012.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['directions_str'] = filtered_data['directions'].dropna().apply(lambda x: " ".join(x))  # Convertir listas a strings


1
Resumen generado: In a small bowl, whisk together mayonnaise, lemon juice, Worcestershire sauce, and salt and pepper to taste; season to taste to taste with more salt and pepper to taste. Add capers and fry until capers are crisp and open like flowers, stirring often, 45 to 60 seconds. Add capers and fry until capers are crisp and open like flowers, stirring often, 45 to 60 seconds.
2
Resumen generado: Pancetta-Stuffed Chicken with Fennel and Pancetta is a classic Italian dish that can be made ahead of time, refrigerated, or eaten on the day of preparation. (If using pancetta, cut it in half and discard the fat.)
3
Resumen generado: Coat a 9x13-inch baking dish with cooking spray. Bake stuffing in dish along side turkey until heated through, about 25 minutes. Uncover and bake until top of stuffing is slightly crisp and golden, about 20 minutes longer. Generously butter 13x9x2-inch baking dish, depending on recipe.
4
Resumen generado: pistachios, almonds, orange zest, carrots, barberr

Your max_length is set to 100, but your input_length is only 57. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)


Resumen generado: Duck with Carrots and Beans in a Glaze Servings: 4 Start to finish: 1 hour, 20 minutes 2 carrots, stemmed, seeded, and sliced into 12 inch pieces 1 yellow bell pepper, stemmed, seeded, and sliced into 12 inch pieces 1 yellow onion, stemmed, seeded, and sliced into 12 inch pieces 4 scallions, stems removed, seeded, and sliced into 12 inch pieces
7
Resumen generado: A tropical twist on a classic cocktail, this punch is made with passion-fruit concentrate, mango nectar, guava nectar, and lime juice from South America, as well as gold rum from the United States, to create a refreshing drink with a tropical twist.
8


KeyboardInterrupt: 

In [19]:
print(filtered_data['directions_summary'])
# Exportar solo la columna 'directions_summary' a un archivo CSV
filtered_data[['directions_summary']].to_csv('resumenes.csv', index=False)

print("La columna 'directions_summary' ha sido exportada a 'resumenes.csv'.")



19        Potatoes and capers can be made 2 hours ahead...
53        Preheat oven to 400°F . Rub each chicken with...
62        Preheat oven to 350°F . Melt butter in heavy ...
83        Preheat oven to 350° . Spread pistachios on a...
105       Set up a grill for indirect grilling with onl...
                               ...                        
20099     Pork can be marinated 4 hours ahead; keep chi...
20109     Toss pears, salt, and 1/2 cup caramel sauce i...
20111     Mayonnaise, lemon juice and garlic blend in s...
20114     Mix whiskey, maple syrup, bitters, and 1 tsp....
20121     Chicken, fennel, shallots, garlic and garlic ...
Name: directions_summary, Length: 1565, dtype: object
La columna 'directions_summary' ha sido exportada a 'resumenes.csv'.


In [15]:
from transformers import AutoTokenizer, pipeline

# Cargar el modelo y el tokenizador
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

# Cargar el summarizer de Hugging Face 
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Preprocesar y contar palabras en 'directions'
filtered_data['desc_str'] = filtered_data['desc'].dropna().apply(lambda x: " ".join(x))  # Convertir lista a string
filtered_data['word_count'] = filtered_data['desc_str'].apply(lambda x: len(x.split()))  # Contar palabras

# Función para resumir o copiar según el número de palabras
def summarize_or_copy(text, word_count, threshold=50):
    if word_count > threshold:
# Tokenizar y truncar el texto
        inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt")
        # Usar el modelo summarizer con el texto tokenizado
        summary = summarizer(tokenizer.decode(inputs['input_ids'][0]), max_length=50, min_length=10, do_sample=False)
        return summary[0]['summary_text']        
    else:
        return text  # Copia el texto original si es corto

# Aplicar la función a la columna 'directions_str'
filtered_data['desc_summary'] = filtered_data.apply(
    lambda row: summarize_or_copy(row['desc_str'], row['word_count']), axis=1
)

# Verificar resultados
print(filtered_data[['desc_str', 'word_count', 'desc_summary']].head(10))

Device set to use cpu
C:\Users\casa\AppData\Local\Temp\ipykernel_17220\2405994670.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['desc_str'] = filtered_data['desc'].dropna().apply(lambda x: " ".join(x))  # Convertir lista a string
C:\Users\casa\AppData\Local\Temp\ipykernel_17220\2405994670.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['word_count'] = filtered_data['desc_str'].apply(lambda x: len(x.split()))  # Contar palabras
Token indices sequence length is longer t

IndexError: index out of range in self